In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 3, Finished, Available)

In [2]:
#Importing all required python libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 4, Finished, Available)

In [ ]:
!pip install kaggle

 #### A notebook created in our workspace to perform extraction of data from kaggle using kaggle api and then data is loaded to the lakehouse before it is being transformed and modeled into a usable format for analysis in Power BI 

In [5]:
#Creating a directory for storage of files and connecting to kaggle api

import os
os.chdir('/lakehouse/default/Files')
os.environ['KAGGLE_USERNAME']='oyeladunoluwatosin'
os.environ['KAGGLE_KEY']='12bfb18e713c1d5cbf8714a7ffc58a37'

from kaggle.api.kaggle_api_extended import KaggleApi
api=KaggleApi()
api.authenticate()
api.dataset_download_file('willianoliveiragibin/illicit-drugs', 'deaths-illicit-drugs NEW.csv')


StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 7, Finished, Available)

Dataset URL: https://www.kaggle.com/datasets/willianoliveiragibin/illicit-drugs


True

In [9]:
#Datafrom file is now read into a pandas dataframe
# File AP path copied
df = pd.read_csv("/lakehouse/default/Files/deaths-illicit-drugs%20NEW.csv")
df.head()

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 11, Finished, Available)

,COUNTRY,Code,Year,Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number),"Deaths that are from all causes attributed to drug use, in both sexes aged all ages"
0,American Samoa,ASM,1990,0,13.090.138
1,Andorra,AND,1990,0,20.659.685
2,Antigua and Barbuda,ATG,1990,0,6.584.844
3,Bahamas,BHS,1990,0,39.783.647
4,Barbados,BRB,1990,0,26.759.732


#### Data Wrangler will be used to clean the data and transform to a format required for modeling. The fact table and dimension tables will be created using the data wrangler tools. The mmodelin type will be a star Schema with one fact table and two dimension tables(Year and Country)

In [10]:
# Wrangling out the first dimension table DimDate
# Code generated by Data Wrangler for pandas DataFrame
# All columns except 'Year' dropped
# Dupliacates removed
# Dataframe renamed

def clean_data(df):
    # Drop columns: 'COUNTRY', 'Code' and 2 other columns
    df = df.drop(columns=['COUNTRY', 'Code', 'Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number)', 'Deaths that are from all causes attributed to drug use, in both sexes aged all ages'])
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    return df

DimDate = clean_data(df.copy())
DimDate.head()

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 34, Finished, Available)

,Year
0,1990
232,1991
464,1992
696,1993
928,1994


In [11]:
# The index is reset
DimDate.reset_index(inplace=True,drop=True)
DimDate.head()

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 35, Finished, Available)

,Year
0,1990
1,1991
2,1992
3,1993
4,1994


In [12]:
# Wrangling out the country dimension table
# Code generated by Data Wrangler for pandas DataFrame
# All columns except 'Country' dropped
# Dupliacates removed
# Dataframe renamed

def clean_data(df):
    # Drop columns: 'Code', 'Year' and 2 other columns
    df = df.drop(columns=['Code', 'Year', 'Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number)', 'Deaths that are from all causes attributed to drug use, in both sexes aged all ages'])
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    return df

DimCountry = clean_data(df.copy())
DimCountry.head()

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 56, Finished, Available)

,COUNTRY
0,American Samoa
1,Andorra
2,Antigua and Barbuda
3,Bahamas
4,Barbados


In [17]:
##Create a country ID for the list of countries
DimCountry['CountryID'] = range(0, len(DimCountry))
DimCountry.head()

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 61, Finished, Available)

,COUNTRY,CountryID
0,American Samoa,0
1,Andorra,1
2,Antigua and Barbuda,2
3,Bahamas,3
4,Barbados,4


In [23]:
# Creating the fact table by dropping country and replacing with the CountryID
df = df.merge(DimCountry, how='left', on='COUNTRY')

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 99, Finished, Available)

In [24]:
# Wrangling the data to drop country code

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 100, Finished, Available)

,COUNTRY,Code,Year,Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number),"Deaths that are from all causes attributed to drug use, in both sexes aged all ages",CountryID
0,American Samoa,ASM,1990,0,13.090.138,0
1,Andorra,AND,1990,0,20.659.685,1
2,Antigua and Barbuda,ATG,1990,0,6.584.844,2
3,Bahamas,BHS,1990,0,39.783.647,3
4,Barbados,BRB,1990,0,26.759.732,4


In [ ]:
# Code generated by Data Wrangler for pandas DataFrame
#Fact Table generated by replacing country with countryID and dropping country code

def clean_data(df):
    # Drop columns: 'COUNTRY', 'Code'
    df = df.drop(columns=['COUNTRY', 'Code'])
    return df

fact = clean_data(df.copy())
fact.head()

StatementMeta(, , , Cancelled, )

In [31]:
# Code generated by Data Wrangler for pandas DataFrame
# Columns renamed

def clean_data(fact):
    # Rename column 'Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number)' to 'DeathsbyDruguseDisorder'
    fact = fact.rename(columns={'Deaths - Drug use disorders - Sex: Both - Age: All Ages (Number)': 'DeathsbyDruguseDisorder'})
    # Rename column 'Deaths that are from all causes attributed to drug use, in both sexes aged all ages' to 'DeathsFromAllCausesDrugUse'
    fact = fact.rename(columns={'Deaths that are from all causes attributed to drug use, in both sexes aged all ages': 'DeathsFromAllCausesDrugUse'})
    return fact

fact = clean_data(fact.copy())
fact.head()

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 163, Finished, Available)

,Year,DeathsbyDruguseDisorder,DeathsFromAllCausesDrugUse,CountryID
0,1990,0,13.090.138,0
1,1990,0,20.659.685,1
2,1990,0,6.584.844,2
3,1990,0,39.783.647,3
4,1990,0,26.759.732,4


#### Now we have the three separate tables required to perform the star schema model. The dataframe will then be loaded into a spark dataframe which is then saved as a table

In [32]:
#Create as spark dataframe
sparkDimDate = spark.createDataFrame(DimDate)
sparkfact = spark.createDataFrame(fact)
sparkDimCountry = spark.createDataFrame(DimCountry)

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 164, Finished, Available)

In [33]:
#save as a table format
sparkDimDate.write.format("delta").mode("Overwrite").saveAsTable("Date")
sparkDimCountry.write.format("delta").mode("Overwrite").saveAsTable("Country")
sparkfact.write.format("delta").mode("Overwrite").saveAsTable("DrugsFacts")

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 165, Finished, Available)

StatementMeta(, 422726c9-ed9e-41de-85a1-dd19aba59ef6, 166, Finished, Available)